In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/"

In [6]:
import pandas as pd
pd.set_option("max_colwidth", None)

## steps
  1. tokenizeing
  2. stop words removal
  3. normalization on the last result

    3.1 on the stem result

    3.2 on the stop words result
  4. stemming two ways:

    4.1 on the normalization result

    4.2 on the stop words result

  -- emojies

## Get data

In [7]:

data_path= path + "ArSAS_dataset.csv"
df = pd.read_csv(data_path)

In [8]:
df.head(20)

,Tweet_text,label,label_name
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,2,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين :#اليمن عدوان وحصار ل 3 سنوات #البحرين قمع حراك شعبها المسالم المطالب بالمساواة والعداله #سوريا #العراق دعموا الإرهاب وارسلوا المال والسلاح والانتحاريين #قطر حصار ومحاولة فرض الوصايه والآن #لبنان محاولة فرض وصايه علني!!,0,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت بارتياح وأنا أتابعه من باريس - https://t.co/hSvsbEaeUz #youm,2,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,0,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 2018 روسيا GHA 0vs1 EYG: https://t.co/9amtlpYR4C,1,Neutral
5,طبعا 25 يناير ثورة محترمة ونفخر بها كان فيها عملاء وخونه واصحاب اجندات ايوة كان فيها زيها زى اى ثورة بيحاول اصحاب المصالح الركوب عليها وده حصل فعلا من الاخوان وتم تصحيح المسار بى 30 / 6 واللى يقول كلام غير ده يبقى كداب لان دى الحقيقة المجردة من اى مصالح او اطماع,3,Mixed
6,#WeNeedToTalk #منتدي #شباب #العالم #شرم_الشيخ #حفل #الختام @ قاعة المؤتمرات بشرم الشيخ https://t.co/lSQhAGm97T,1,Neutral
7,يعنى رايح تعتقل ماهيتور المصرى تانى بحجة اشتراكها فى مظاهرة بمصرية تيران وصنافير من 6 شهور ! هو يعنى سجلك الزفت... https://t.co/LrzuXGTi,0,Negative
8,#قولوا_ل_قطر كبتن نامق أن الحل ب #الرياض وان ما يحدث لقطر من مقاطعة هي لعدم استجابتها لاتفاق #الرياض. وان الذي يحدث الآن للبنان شان لبناني داخلي صرف. وهل أصبح #نامق المتحدث الرسمي لايران بلسان #عربي وأول مرة ارى وزير دولة يصف دولته بالساندويتش #قطر_ساندويتش https://t.co/9KBKDJuVsb,0,Negative
9,بسم الله نبدأ 👏 نغرد علي وسم 👇 👇 👇 👇 👇 ↩ #شباب_مصر_يريد ↪تحقيق أهداف ثورة يناير العيش والحرية والعدالة الاجتماعية والكرامة الإنسانية #تيم_المرابطون ✊ https://t.co/RDDVULxYuK,2,Positive


## Data Cleaning

In [9]:
### Define function to remove arabic puctuations as well english puncts
import re
import string

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [10]:
def clean_arabic_text(text):

    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove hashtags and tags
    text = re.sub(r'#\w+\s*', '', text)
    text = re.sub(r'@\w+\s*', '', text)
    text = re.sub(r'[a-zA-Z0-9]+', '', text) # remove numbers and non arabic characters
    text = re.sub(r'(.)\1+', r'\1', text) # Elongation removal
    text = remove_punctuations(text)
    # Remove emojies
    text = re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+', ' ', text)
    # Remove extra spaces
    text = re.sub(r'[\n\t\ ]+', ' ', text)

    return text

In [11]:
df['clean_text_withEmojies'] = df['Tweet_text'].apply(clean_arabic_text)

In [15]:
df['clean_text'] = df['Tweet_text'].apply(clean_arabic_text)

In [16]:
df[['Tweet_text',	'label',	'clean_text_withEmojies', 'clean_text',]]

,Tweet_text,label,clean_text_withEmojies,clean_text
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,2,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين :#اليمن عدوان وحصار ل 3 سنوات #البحرين قمع حراك شعبها المسالم المطالب بالمساواة والعداله #سوريا #العراق دعموا الإرهاب وارسلوا المال والسلاح والانتحاريين #قطر حصار ومحاولة فرض الوصايه والآن #لبنان محاولة فرض وصايه علني!!,0,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت بارتياح وأنا أتابعه من باريس - https://t.co/hSvsbEaeUz #youm,2,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس
3,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,0,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 2018 روسيا GHA 0vs1 EYG: https://t.co/9amtlpYR4C,1,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا
...,...,...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 العالم https://t.co/AQF93ksvJo #تصفيات_كأس_العالم #الجزائر_نيجيريا,1,ملخص مباراة نيجيريا الجزائر تصفيات كأس العالم,ملخص مباراة نيجيريا الجزائر تصفيات كأس العالم
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز 11 تعادل بدون أي خسارة سواء رسمياً أو ودياً ! آخر تأهل من الملحق كان في نفس الملعب 1998 * ايطاليا المنتخب الوحيد عبر التاريخ الذي لم يخسر على أرضه في كأس العالم سواء تصفيات أو نهائيات . كمية تفاؤل كبيرة #الإياب_الإنتحاري https://t.co/qtIQrpC1gL,2,ايطاليا في السان سيرو لعبت مباراة فوز تعادل بدون أي خسارة سواء رسمياً أو ودياً آخر تأهل من الملحق كان في نفس الملعب ايطاليا المنتخب الوحيد عبر التاريخ الذي لم يخسر على أرضه في كأس العالم سواء تصفيات أو نهائيات كمية تفاؤل كبيرة,ايطاليا في السان سيرو لعبت مباراة فوز تعادل بدون أي خسارة سواء رسمياً أو ودياً آخر تأهل من الملحق كان في نفس الملعب ايطاليا المنتخب الوحيد عبر التاريخ الذي لم يخسر على أرضه في كأس العالم سواء تصفيات أو نهائيات كمية تفاؤل كبيرة
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة الذهاب إيرلندا0️⃣ ❌ سويسرا1️⃣ مشاهدة الهدف📺 https://t.co/MyxbCrRFVR #تصفيات_كأس_العالم,1,المُلحق المؤهل لكأس العالم أوروبا مباراة الذهاب إيرلندا سويسرا مشاهدة الهدف,المُلحق المؤهل لكأس العالم أوروبا مباراة الذهاب إيرلندا سويسرا مشاهدة الهدف
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله المغازي #ONLIVE | #صباح_ON https://t.co/PlTnuPWGO,1,رسائل وتوصيات منتدى شباب العالم د عبداله المغازي,رسائل وتوصيات منتدى شباب العالم د عبداله المغازي


In [ ]:
# df.to_csv(path +'cleaned_text.csv', index=False)

## Tokenizing & Stop words removal

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

stop_words = set(stopwords.words('arabic'))

def remove_stopwords(text):
    tokens = nltk.word_tokenize(text) # tokenizing
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    filtered_text = re.sub('\\s+', ' ', text) # remove extra space due to stop words removal
    return filtered_text

df['noStopWords'] = df['clean_text'].apply(remove_stopwords)

#df.to_csv('/content/drive/MyDrive/dataWithToken.csv', index=False)

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Tweet_text,label,label_name,clean_text_withEmojies,clean_text,noStopWords
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,2,Positive,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين :#اليمن عدوان وحصار ل 3 سنوات #البحرين قمع حراك شعبها المسالم المطالب بالمساواة والعداله #سوريا #العراق دعموا الإرهاب وارسلوا المال والسلاح والانتحاريين #قطر حصار ومحاولة فرض الوصايه والآن #لبنان محاولة فرض وصايه علني!!,0,Negative,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت بارتياح وأنا أتابعه من باريس - https://t.co/hSvsbEaeUz #youm,2,Positive,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس
3,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,0,Negative,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 2018 روسيا GHA 0vs1 EYG: https://t.co/9amtlpYR4C,1,Neutral,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا


In [ ]:
# df[['Tweet_text', 'label','noStopWords']].to_csv(path + 'data_noStopWords.csv', index=False, encoding='utf-8')

## Normalization before stemming

In [22]:
!pip install pyarabic

In [23]:
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import strip_tatweel
from pyarabic.araby import normalize_alef
from pyarabic.araby import normalize_hamza
from pyarabic.araby import normalize_teh

def normalization(text):
        normalized_text = araby.strip_tatweel(text)
        normalized_text = araby.strip_tashkeel(normalized_text)
        normalized_text = araby.strip_harakat(normalized_text)
        normalized_text = araby.normalize_hamza(normalized_text)
        normalized_text = araby.normalize_alef(normalized_text)   # Alef variations to plain a Alef character
        normalized_text = araby.normalize_hamza(normalized_text)
        normalized_text = araby.normalize_teh(normalized_text)     # Teh Marbuta characters to a Heh character
        normalized_text = araby.strip_tatweel(normalized_text)
        return normalized_text

df['normalized_text']= df['noStopWords'].apply(normalization)
#df.to_csv('/content/drive/MyDrive/dataWithNormalized.csv', index=False)
df.head()

## Stemming

### farasa stemmer

In [ ]:
#Stemming
!pip install -U farasapy
from farasa.stemmer import FarasaStemmer
from farasa.segmenter import FarasaSegmenter

# create a stemmer object
farasa_stemmer = FarasaStemmer(interactive=True)

# define a function to apply stemming on a text column
def stem_text(text):
    stemmed_text = farasa_stemmer.stem(text)
    return stemmed_text

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:20<00:00, 12.0MiB/s]


[2023-07-05 09:51:34,813 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


#### Apply stemming before normalization

In [ ]:
stemm_farsa = []
for sample in df['noStopWords'].values:
    stemmed = farasa_stemmer.stem(sample)
    stemm_farsa.append(stemmed)

In [ ]:
df['stemmed_farasa'] = stemm_farsa

#### Apply normalization after stemming

In [ ]:
df['stemm_farasa_norm']= df['stemmed_farasa'].apply(normalization)

#### Apply stemming after normalization

In [ ]:
norm_stemm_farasa = []
for sample in df['normalized_text'].values:
    norm_stemm_fr = farasa_stemmer.stem(sample)
    norm_stemm_farasa.append(norm_stemm_fr)

In [ ]:
df['norm_stemm_farasa'] = norm_stemm_farasa

### ISRI stemmer

In [ ]:
import nltk
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()

#### Apply ISRI stemmer before normalization

In [ ]:
stemmed_text_ISRI = []
for smpl in df['noStopWords'].values:
    stemmedIS = st.stem(smpl)
    stemmed_text_ISRI.append(stemmedIS)

In [ ]:
df['stemmed_ISRI'] = stemmed_text_ISRI
df[['noStopWords', 'stemmed_ISRI']].head()

,noStopWords,stemmed_ISRI
0,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,مباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه عل
2,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس
3,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن
4,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا,اهداف مباراة غانا مصر تصفيات كأس العالم روسيا


#### Apply normalization after stemming

In [ ]:
df['stemm_ISRI_norm']= df['stemmed_ISRI'].apply(normalization)

#### Apply ISRI stemmer After normalization

In [ ]:
norm_stemmISRI = []
for sample in df['normalized_text'].values:
    stemmed = st.stem(sample)
    norm_stemmISRI.append(stemmed)

In [ ]:
df['norm_stemm_ISRI'] = norm_stemmISRI
df[['noStopWords', 'normalized_text', 'norm_stemm_ISRI']].head()

,noStopWords,normalized_text,norm_stemm_ISRI
0,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراه القادمه الجوله الءخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التءهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش,مباراه القادمه الجوله الءخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التءهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسه خارجيه لدوله تحترم نفسها والءءخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواه والعداله دعموا الءرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاوله فرض الوصايه والءءن محاوله فرض وصايه علني,هل هذه هي سياسه خارجيه لدوله تحترم نفسها والءءخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواه والعداله دعموا الءرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاوله فرض الوصايه والءءن محاوله فرض وصايه عل
2,وزير خارجية فرنسا عن منتدى شباب العالم شعرت بارتياح وأنا أتابعه من باريس,وزير خارجيه فرنسا عن منتدا شباب العالم شعرت بارتياح وءنا ءتابعه من باريس,وزير خارجيه فرنسا عن منتدا شباب العالم شعرت بارتياح وءنا ءتابعه من باريس
3,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض إلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض ءلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن,ومع السيسي و بشار و ايران و بن زايد و والا خليفه و روافض ءلعراق و حفتر و علي صالح كل طواغيت العرب العلاقات عسل علي سمن
4,أهداف مباراة غانا مصر تصفيات كأس العالم روسيا,ءهداف مباراه غانا مصر تصفيات كءس العالم روسيا,ءهداف مباراه غانا مصر تصفيات كءس العالم روسيا


## Save all processed steps to csv

In [ ]:
df.columns

Index(['Tweet_text', 'label', 'label_name', 'clean_text_withEmojies',
       'clean_text', 'noStopWords', 'normalized_text', 'stemmed_farasa',
       'stemm_farasa_norm', 'norm_stemm_farasa', 'stemmed_ISRI',
       'stemm_ISRI_norm', 'norm_stemm_ISRI'],
      dtype='object')

In [ ]:
import csv
df.to_csv(path + 'preprocessedData.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

In [ ]:
data = path + 'preprocessedData.csv'
preprocessedData_df = pd.read_csv(data, encoding='utf-8')


In [ ]:

preprocessedData_df.head(30)

,Tweet_text,label,label_name,clean_text_withEmojies,clean_text,noStopWords,normalized_text,stemmed_farasa,stemm_farasa_norm,norm_stemm_farasa,stemmed_ISRI,stemm_ISRI_norm,norm_stemm_ISRI
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,2,Positive,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش 😀😁,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,المباراه القادمه الجوله الءخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التءهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش,مباراة قادم جولة أخير من مجموعة ال تصفية كاس عالم روسيا ترتيب مصر مركز أول نقطة تم حسم تأهل مونديال غد ساعة ع قناة بين سبورت توقع ل نتيجة ماتش,مباراه قادم جوله ءخير من مجموعه ال تصفيه كاس عالم روسيا ترتيب مصر مركز ءول نقطه تم حسم تءهل مونديال غد ساعه ع قناه بين سبورت توقع ل نتيجه ماتش,المباراه قادم جوله ءخيره من مجموع ال تصفية كاس عالم روسيا ترتيب مصر مركز أول نقطة تم حسم تءهل مونديال غد ساعه ع قنا بين سبورت توقع ل نتيج ماتش,مباراة القادمة الجولة الأخيرة من المجموعة ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطة تم حسم التأهل لمونديال غدا الساعة ع قناة بين سبورت توقعاتكم ل نتيجة الماتش,مباراه القادمه الجوله الءخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التءهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش,مباراه القادمه الجوله الءخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التءهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين :#اليمن عدوان وحصار ل 3 سنوات #البحرين قمع حراك شعبها المسالم المطالب بالمساواة والعداله #سوريا #العراق دعموا الإرهاب وارسلوا المال والسلاح والانتحاريين #قطر حصار ومحاولة فرض الوصايه والآن #لبنان محاولة فرض وصايه علني!!,0,Negative,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وصايه علني,هل هذه هي سياسه خارجيه لدوله تحترم نفسها والءءخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواه والعداله دعموا الءرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاوله فرض الوصايه والءءن محاوله فرض وصايه علني,هل هذا هي سياسة خارجي دولة احترم نفس آخر عدوان حصار ل سنة قمع حراك شعب مسالم مطلب مساواة عداله دعم إرهاب ارسل مال سلاح انتحار حصار محاولة فرض وصايه الآن محاولة فرض وصاي علني,هل هذا هي سياسه خارجي دوله احترم نفس ءءخر عدوان حصار ل سنه قمع حراك شعب مسالم مطلب مساواه عداله دعم ءرهاب ارسل مال سلاح انتحار حصار محاوله فرض وصايه الءءن محاوله فرض وصاي علني,هل هذا هي سياس خارجي دولة احترم نفس ءءخر عدوان حصار ل سنة قمع حراك شعب مسالم مطلب مساواه عداله دعم ءرهاب ارسل مال سلاح انتحار حصار محاول فرض وصايه ءءن محاول فرض وصاي علني,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخرين عدوان وحصار ل سنوات قمع حراك شعبها المسالم المطالب بالمساواة والعداله دعموا الإرهاب وارسلوا المال والسلاح والانتحارين حصار ومحاولة فرض الوصايه والآن محاولة فرض وص